## Solving Tic-Tac-Toe using TD(λ)
In this assignment you will build an RL agent capable of playing Tic-Tac-Toe using TD(λ) algorithm and the environment simulated by you in first week.

First of all copy the environment simulated by you from the first week below.
- Note that you should also return the state of the board each time you call act function, ideally the state should be stored in a numpy array for faster implementation
- The only input the function can take is via its own arguments and not input() function.

The ideal TicTacToe environment:
- Will take N, the size of board as an argument in its constructor.
- Will have act function taking a single argument representing the action taken (preferably int type) and return the state of board (preferably numpy array), reward signal (float) and bool value "done" which is true if the game is over else false.
- Will have reset function which resets the board and starts a new game.
- Will give reward signal as 1 if 'X' won and -1 if 'O' won (or vice-versa) and 0 if its a draw.
- Will take alternate calls of act function as the moves of one player.

For example:
```html
env.reset()
Returns ==> (array([[0., 0., 0.],[0., 0., 0.],[0., 0., 0.]]), 0, False)
                | | | |
        Board:  | | | |
                | | | |

env.act(4)
Returns ==> (array([[0., 0., 0.],[0., 1.0, 0.],[0., 0., 0.]]), 0, False)
                | | | |
        Board:  | |X| |
                | | | |

env.act(0)
Returns ==> (array([[-1.0, 0., 0.],[0., 1.0, 0.],[0., 0., 0.]]), 0, False)
                |O| | |
        Board:  | |X| |
                | | | |

env.act(7)
Returns ==> (array([[-1.0, 0., 0.],[0., 1.0, 0.],[0., 1.0, 0.]]), 0, False)
                |O| | |
        Board:  | |X| |
                | |X| |

env.act(6)
Returns ==> (array([[-1.0, 0., 0.],[0., 1.0, 0.],[-1.0, 1.0, 0.]]), 0, False)
                |O| | |
        Board:  | |X| |
                |O|X| |

env.act(2)
Returns ==> (array([[-1.0, 1.0, 0.],[0., 1.0, 0.],[-1.0, 1.0, 0.]]), 1, True)
                |O|X| |
        Board:  | |X| |
                |O|X| |


```
<hr>

Note : You can change your TicTacToe environment code before using it here


In [ ]:
# Import any necessary libraries here
import numpy as np
from tqdm import tqdm
import random

Your TicTacToe environment class comes here

In [ ]:
class TicTacToe:
    def __init__(self,N):
        self.N=N
        self.board = np.zeros((self.N,self.N))
        self.player = 'X'
        self.winner = ''
        self.game_complete = False
    # Given below is the preferable structure of act function
    def act(self, action : int) -> tuple:     # Returns tuple of types (np.ndarray, int, bool)
        row = (action)//(self.N)
        col = (action)%(self.N)
        if self.game_complete==False and self.board[row][col]==0:
            if self.player=='X':
                self.board[row][col] = 1
            elif self.player=='O':
                self.board[row][col] = -1
            self._winnercheck()
            self._drawcheck()
            if self.player=='X':
                self.player='O'
            elif self.player=='O':
                self.player='X'
            #print('Move Succesful')
            for i in self.board:
                s=''
                for j in i:
                    if j==1:
                        s+='X'
                    elif j==-1:
                        s+='O'
                    else:
                        s+=' '
                s="|" + "|".join(s) + "|"
                print(s)
            reward=0
            if self.game_complete==True:
                if self.winner=='X':
                    reward=1
                elif self.winner=='O':
                    reward=-1
            return (self.board,reward,self.game_complete)
        else:
            print('Invalid Move!')
            return

    def reset(self):
        self.board =np.zeros((3,3))
        self.player = 'X'
        self.winner = ''
        self.game_complete = False
        reward=0
        return (self.board,reward,self.game_complete)
    
    def _winnercheck(self):
        temp1=False
        for i in range(self.N):
            if self.board[i][0]!=0 and len(np.unique(self.board[i]))==1:
                temp1=True
        temp2=False
        for i in range(self.N):
            if self.board.T[i][0]!=0 and len(np.unique(self.board.T[i]))==1:
                temp2=True
        if temp1 or temp2:
            self.game_complete = True
            self.winner = self.player
            return True
        temp3=False
        if self.board[0][0]!=0:
            for i in range(1, self.N):
                if self.board[i][i] != self.board[0][0]:
                    break
            else:
                temp3=True
        
        if self.board[self.N-1][0]!=0:
            for i in range(1, self.N):
                if self.board[self.N-i-1][i] != self.board[self.N-1][0]:
                    break
            else:
                temp3=True
        if temp3:
            self.game_complete=True
            self.winner=self.player
            #print("hi")
            return True
    def _drawcheck(self):
        if self.game_complete==False and (0 not in self.board):
            self.game_complete=True
            self.winner= 'Draw'

In [ ]:
env = TicTacToe(3)

Then comes the agent class which
- Uses TD(λ) algorithm to find the optimal policies for each state
- Stores the calculated optimal policies as a .npy file for later use
- Calculates the average return of the itself against a random player (makes random moves on its chance) periodically during training and plot it (for example if total training iterations is 10000, then calculate average return after each 500 steps, also for average return you should calculate return atleast 5 times and then take average)
- You can make additional functions

You can store all the encountered states in a numpy array (which will have 3 dims) and then store corresponding values for that particulare state in another array (will have 1 dims) and then you can store all these arrays in a .npy file for future use, so that you don't have to train the model each time you want to play TicTacToe

In [ ]:
class TicTacToeagent:
    def __init__(self, env):
        self.env = env

    def decay_schedule(self, init_value, min_value, decay_ratio, max_steps, log_start=-2, log_base=10):
        decay_steps = int(max_steps * decay_ratio)
        rem_steps = max_steps - decay_steps
        values = np.logspace(log_start, 0, decay_steps, base=log_base, endpoint=True)[::-1]
        values = (values - values.min()) / (values.max() - values.min())
        values = (init_value - min_value) * values + min_value
        values = np.pad(values, (0, rem_steps), 'edge')
        return values

    
    def train(self, gamma=0.9, init_alpha=0.1, min_alpha=0.01, alpha_decay_ratio=0.8,
              init_epsilon=1.0, min_epsilon=0.1, epsilon_decay_ratio=0.9, n_episodes=10000):
        nS, nA = 3 ** (self.env.N ** 2), (self.env.N ** 2)

        Q = np.random.rand(nS, nA)
        states=list()
        def select_action(state_code, Q, epsilon, actions_possible):
            if (np.random.random() > epsilon) and (np.argmax(Q[state_code]) in actions_possible):
                a = np.argmax(Q[state_code])
            else:
                a = random.choice(actions_possible)
            actions_possible.remove(a)
            return a
       

        initial_state = np.zeros((3,3))
        temp=tuple(initial_state.tolist())
        states.append(temp)

        alphas = self.decay_schedule(init_alpha, min_alpha, alpha_decay_ratio, n_episodes)
        epsilons = self.decay_schedule(init_epsilon, min_epsilon, epsilon_decay_ratio, n_episodes)

        for e in tqdm(range(n_episodes), leave=False):
            state, _, done = self.env.reset()
            
            actions_possible = list(range(self.env.N ** 2))
            print(actions_possible,"hi")
            temp=tuple(state.tolist())
            state_code = states.index(temp)
            action = select_action(state_code, Q, epsilons[e], actions_possible)

            while not done:
                print(actions_possible,"hi2")
                #print(action)
                next_state, reward, done = self.env.act(action)
                #print(next_state, done)
                temp=tuple(next_state.tolist())
                if temp not in states:
                    states.append(temp)
                    
                temp=tuple(next_state.tolist())
                next_state_code = states.index(temp)
                if done:
                    next_action = random.choice(list(range(self.env.N ** 2)))
                else:
                    next_action = select_action(next_state_code, Q, epsilons[e], actions_possible)
                print(actions_possible,"hi3")
                temp=tuple(state.tolist())
                state_code = states.index(temp)
                td_target = reward + (gamma*Q[next_state_code][next_action])
                td_error = td_target - Q[state_code][action]

                Q[state_code][action] += (alphas[e] * td_error)

                state, action = next_state, next_action

        V = np.max(Q, axis=1)
        pi = lambda s: {s: a for s, a in enumerate(np.argmax(Q, axis=1))}[s]
        return Q, V, pi, states

Now for evaluation purposes and for your self checking the code block below after running should:
- Initialize the agent and call the train function which trains the agent
- Load the stored state value data
- Start a single player game of TicTacToe which takes input from the user for moves according to the convention given below, where the trained Q values play as computer

In [ ]:
agent = TicTacToeagent(env)

Q_values, values, policy, states = agent.train()


In [ ]:
state_codes = range(len(states))  # Replace with the actual states in your environment
policy_values = {i: policy(i) for i in state_codes}
np.save('policy_values.npy', policy_values)
loaded_policy_values = np.load('policy_values.npy', allow_pickle=True).item()

In [ ]:
print("1 | 2 | 3")
print("4 | 5 | 6")
print("7 | 8 | 9")
'''The model should train a 3x3 TicTacToe by default, you can definitely modify the values(of N, number of iterations etc) for your convenience but training model for bigger N might take lot of time
'''

# Code Here
def play():
    def print_board(board):
        for row in board:
            s = ''
            for cell in row:
                if cell == 1:
                    s += 'X'
                elif cell == -1:
                    s += 'O'
                else:
                    s += ' '
            print("|" + "|".join(s) + "|")

    state, _, done = env.reset()
    while not done:
        print("\nCurrent Board:")
        print_board(state)

        if env.player == 'O':
            action = int(input("Enter your move (1-9): ")) - 1
        else:
            temp = tuple(state.tolist())
            state_code = states.index(temp)
            action = policy(state_code)
            print(action + 1)  # Adding 1 to match the user input

        next_state, reward, done = env.act(action)

        if not done:
            state = next_state
        else:
            print("\nGame Over!")
            if reward == 1:
                print("Computer won!")
            elif reward == -1:
                print("You Won!")
            else:
                print("It's a draw!")

# Call the play function
play()

loaded_policy_values